In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.base import TransformerMixin, BaseEstimator

import matplotlib.pyplot as plt
from matplotlib import ticker
# unused but required import for doing 3d projections with matplotlib < 3.2
import mpl_toolkits.mplot3d  # noqa: F401
from sklearn import manifold

from category_encoders.target_encoder import TargetEncoder
from imblearn.under_sampling import NearMiss
from sklearn.metrics import f1_score

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.feature_selection import SelectFromModel

from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay, classification_report

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

random_state = 42

import warnings
warnings.filterwarnings("ignore")

In [2]:
raw_data = pd.merge(pd.read_csv('../train_transaction.csv'), pd.read_csv('../train_identity.csv'),
                    left_on='TransactionID', right_on='TransactionID', how='left')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
raw_data.describe()

TransactionID        isFraud  TransactionDT  TransactionAmt  \
count   5.905400e+05  590540.000000   5.905400e+05   590540.000000   
mean    3.282270e+06       0.034990   7.372311e+06      135.027176   
std     1.704744e+05       0.183755   4.617224e+06      239.162522   
min     2.987000e+06       0.000000   8.640000e+04        0.251000   
25%     3.134635e+06       0.000000   3.027058e+06       43.321000   
50%     3.282270e+06       0.000000   7.306528e+06       68.769000   
75%     3.429904e+06       0.000000   1.124662e+07      125.000000   
max     3.577539e+06       1.000000   1.581113e+07    31937.391000   

               card1          card2          card3          card5  \
count  590540.000000  581607.000000  588975.000000  586281.000000   
mean     9898.734658     362.555488     153.194925     199.278897   
std      4901.170153     157.793246      11.336444      41.244453   
min      1000.000000     100.000000     100.000000     100.000000   
25%      6019.000000     214.000000     150.000000     166.000000   
50%      9678.000000     361.000000     150.000000     226.000000   
75%     14184.000000     512.000000     150.000000     226.000000   
max     18396.000000     600.000000     231.000000     237.000000   

               addr1          addr2          dist1         dist2  \
count  524834.000000  524834.000000  238269.000000  37627.000000   
mean      290.733794      86.800630     118.502180    231.855423   
std       101.741072       2.690623     371.872026    529.053494   
min       100.000000      10.000000       0.000000      0.000000   
25%       204.000000      87.000000       3.000000      7.000000   
50%       299.000000      87.000000       8.000000     37.000000   
75%       330.000000      87.000000      24.000000    206.000000   
max       540.000000     102.000000   10286.000000  11623.000000   

                  C1             C2             C3             C4  \
count  590540.000000  590540.000000  590540.000000  590540.000000   
mean       14.092458      15.269734       0.005644       4.092185   
std       133.569018     154.668899       0.150536      68.848459   
min         0.000000       0.000000       0.000000       0.000000   
25%         1.000000       1.000000       0.000000       0.000000   
50%         1.000000       1.000000       0.000000       0.000000   
75%         3.000000       3.000000       0.000000       0.000000   
max      4685.000000    5691.000000      26.000000    2253.000000   

                  C5             C6             C7             C8  \
count  590540.000000  590540.000000  590540.000000  590540.000000   
mean        5.571526       9.071082       2.848478       5.144574   
std        25.786976      71.508467      61.727304      95.378574   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       1.000000       0.000000       0.000000   
50%         0.000000       1.000000       0.000000       0.000000   
75%         1.000000       2.000000       0.000000       0.000000   
max       349.000000    2253.000000    2255.000000    3331.000000   

                  C9            C10            C11            C12  \
count  590540.000000  590540.000000  590540.000000  590540.000000   
mean        4.480240       5.240343      10.241521       4.076227   
std        16.674897      95.581443      94.336292      86.666218   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       1.000000       0.000000   
50%         1.000000       0.000000       1.000000       0.000000   
75%         2.000000       0.000000       2.000000       0.000000   
max       210.000000    3257.000000    3188.000000    3188.000000   

                 C13            C14             D1             D2  \
count  590540.000000  590540.000000  589271.000000  309743.000000   
mean       32.539918       8.295215      94.347568     169.563231   
std       129.364844      49.544262     157.660387     177.315865   

In [5]:
raw_data.describe(include='object')

ProductCD   card4   card6 P_emaildomain R_emaildomain      M1      M2  \
count     590540  588963  588969        496084        137291  319440  319440   
unique         5       4       4            59            60       2       2   
top            W    visa   debit     gmail.com     gmail.com       T       T   
freq      439670  384767  439938        228355         57147  319415  285468   

            M3      M4      M5      M6      M7      M8      M9     id_12  \
count   319440  309096  240058  421180  244275  244288  244288    144233   
unique       2       3       2       2       2       2       2         2   
top          T      M0       F       F       F       F       T  NotFound   
freq    251731  196405  132491  227856  211374  155251  205656    123025   

         id_15   id_16                 id_23  id_27   id_28   id_29  \
count   140985  129340                  5169   5169  140978  140978   
unique       3       2                     3      2       2       2   
top      Found   Found  IP_PROXY:TRANSPARENT  Found   Found   Found   
freq     67728   66324                  3489   5155   76232   74926   

             id_30        id_31      id_33           id_34   id_35   id_36  \
count        77565       140282      73289           77805  140985  140985   
unique          75          130        260               4       2       2   
top     Windows 10  chrome 63.0  1920x1080  match_status:2       T       F   
freq         21155        22000      16874           60011   77814  134066   

         id_37   id_38 DeviceType DeviceInfo  
count   140985  140985     140810     118666  
unique       2       2          2       1786  
top          T       F    desktop    Windows  
freq    110452   73922      85165      47722

### Разделим данные на train и test

In [3]:
X_train, X_test = train_test_split(raw_data, test_size=0.25, random_state=random_state)
X_train.reset_index(drop=True, inplace=True)
y_train = X_train['isFraud']
X_train.drop(columns=['isFraud', 'TransactionID'], inplace=True)

X_test.reset_index(drop=True, inplace=True)
y_test = X_test['isFraud']
X_test.drop(columns=['isFraud', 'TransactionID'], inplace=True)

In [4]:
del raw_data

### Колонки с большим количеством пропусков

NanFeatureSelector удаляет колонки, в которых доля пропусков больше max_nan_rate (в нашем случае зададим max_nan_rate = 85%)

In [5]:
class NanFeatureSelector(TransformerMixin, BaseEstimator):

    def __init__(self, max_nan_rate):
        self.cols_to_remove = []
        self.max_nan_rate = max_nan_rate

    def fit(self, X, y=None):
        nan_stat = self.get_share_of_NaN(X)
        for i in range(len(nan_stat)):
            column = nan_stat.loc[i]
            if(column['Share_of_NaN'] > self.max_nan_rate):
                self.cols_to_remove.append(column['Name'])
        return self

    def transform(self, X):
        return X.drop(columns=self.cols_to_remove)
    
    def get_share_of_NaN(self, df):
        '''
        Рассчитывает количество пропусков в каждой колонке, а также долю пропусков в каждой колонке
        Параметры:
        df - датафрейм
        Возвращает датафрейм, содержащий информацию о пропусках в каждой колонке датафрейма df
        '''
        result = pd.DataFrame(columns=['Name', 'Number_of_NaN', 'Share_of_NaN'])
        colcount = df.count()
        length = len(df)
        for col_name in colcount.keys():
            result.loc[len(result)] = [col_name, length-colcount[col_name], (length-colcount[col_name])/length]
        return result

### Колонки с большой корреляцией

CorrFeatureSelector удаляет колонки, у которых корреляция больше max_corr (в нашем случае зададим max_corr = 0.9)


In [6]:
class CorrFeatureSelector(TransformerMixin, BaseEstimator):

    def __init__(self, max_corr):
        self.cols_to_remove = set()
        self.max_corr = max_corr

    def fit(self, X, y=None):
        corrs = X.corr()
        cols = corrs.columns
        for i in range(len(cols)):
            col_name_1 = cols[i]
            if col_name_1 in {'TransactionID', 'isFraud', 'TransactionDT'} or col_name_1 in self.cols_to_remove:
                continue
            for j in range(i+1, len(cols)):
                col_name_2 = cols[j]
                if abs(corrs[col_name_1][col_name_2]) > self.max_corr:
                    self.cols_to_remove.add(col_name_2)
        return self

    def transform(self, X):
        return X.drop(columns=self.cols_to_remove)

### Заполнение пропусков

In [7]:
class CustomImputer(TransformerMixin, BaseEstimator):
    
    def __init__(self, strategy='most_frequent'):
        self.__imputer = SimpleImputer(strategy=strategy)
        self.strategy = strategy
        
    def fit(self, X, y=None):
        self.__imputer.fit(X)
        return self
    def transform(self, X, y=None):
        X = X.copy()
        X[X.columns] = self.__imputer.transform(X[X.columns])
        
        return X

### Кодирование категориальных признаков
Признаки, у которых количество уникальных значений больше, чем ohe_limit, закодируем с помощью TargetEncoder. Признаки, у которых количество уникальных значений меньше либо равно ohe_limit, закодируем с помощью pd.get_dummies (One-Hot Encoding).
В нашем случае зададим ohe_limit=10

In [8]:
class ObjectEncoder:
    def __init__(self, ohe_limit, obj_cols):
        self.__ohe_limit = ohe_limit
        self.__obj_cols = obj_cols
        self.__encoded_columns = None
        self.__ohe_cols = []
        self.__targ_enc_cols = []
        self.__targ_enc = None
        
    
    def divide_columns(self, df):
        for col in self.__obj_cols:
            if col in df.columns:
                if df[col].unique().shape[0] > self.__ohe_limit:
                    self.__targ_enc_cols.append(col)
                else:
                    self.__ohe_cols.append(col)
    
    def encode_ohe_cols(self, df):
        # use OHE for columns from self.__ohe_cols only
        if self.__encoded_columns is not None:
            df = pd.get_dummies(df, drop_first=False, columns=self.__ohe_cols)

            removed_cols = []
            for col in df.columns:
                if col not in self.__encoded_columns:
                    removed_cols.append(col)
            df.drop(columns=removed_cols, inplace=True)

            for i in range(len(self.__encoded_columns)):
                col = self.__encoded_columns[i]
                if col not in df:
                    df.insert(i, col, 0)

        else:
            df = pd.get_dummies(df, drop_first=True, columns=self.__ohe_cols)
            self.__encoded_columns = df.columns
            
        return df

    def fit(self, X, y):
        self.divide_columns(X)
        self.__targ_enc = TargetEncoder(cols=self.__targ_enc_cols)
        self.__targ_enc.fit(X, y)
    
    def transform(self, X):
        X = self.__targ_enc.transform(X)
        return self.encode_ohe_cols(X)
    
    def fit_transform(self, X, y):
        self.divide_columns(X)
        self.__targ_enc = TargetEncoder(cols=self.__targ_enc_cols)
        X = self.__targ_enc.fit_transform(X, y)
        return self.encode_ohe_cols(X)

### Масштабирование числовых признаков

In [9]:
class CustomScaler(TransformerMixin, BaseEstimator):
    
    def __init__(self, cols, scaler=None):
        self.cols = cols
        self.scaler = scaler or preprocessing.MinMaxScaler()
        
    def fit(self, X, y=None):
        self.cols = list(set(self.cols).intersection(set(X.columns)))
        num_cols = X[self.cols]
        self.scaler.fit(num_cols)
        return self
    def transform(self, X, y=None):
        X_res = X.copy()
        X_res[self.cols] = self.scaler.transform(X_res[self.cols])
        return X_res

### Категориальные признаки

In [10]:
cat_cols = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
            'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain',
            'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
            'DeviceType', 'DeviceInfo', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20'
            'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32'
            'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

### Числовые признаки

In [11]:
real_cols = list(X_train.columns)

for col in cat_cols:
    if col in real_cols:
        real_cols.remove(col)

### Обучение модели

In [12]:
# Пайплайн заполняет пропуски, кодирует категориальные признаки, масштабирует числовые признаки, выполняет отбор признаков
prep_data_pipe = Pipeline([
    ('nan_feature_selector_', NanFeatureSelector(0.85)),
    ('corr_feature_selector_', CorrFeatureSelector(0.9)),
    ('imputer_', CustomImputer()),
    ('encoder_', ObjectEncoder(10, cat_cols)),
#     ('scaler_', CustomScaler(real_cols, preprocessing.RobustScaler())),
     # MinMaxScaler даёт roc_auc_score лучше, чем RobustScaler
    ('scaler_', CustomScaler(real_cols, preprocessing.MinMaxScaler())), 
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), threshold=5e-2))
])

Посмотрим, какие признаки остаются до того, как SelectFromModel выполнит отбор признаков

In [13]:
prep_data_pipe_1 = prep_data_pipe[:-1]
prep_data_pipe_1

Pipeline(steps=[('nan_feature_selector_',
                 NanFeatureSelector(max_nan_rate=0.85)),
                ('corr_feature_selector_', CorrFeatureSelector(max_corr=0.9)),
                ('imputer_', CustomImputer()),
                ('encoder_',
                 <__main__.ObjectEncoder object at 0x000002662EDC4AC0>),
                ('scaler_',
                 CustomScaler(cols=['TransactionDT', 'TransactionAmt', 'dist1',
                                    'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6',
                                    'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
                                    'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5',
                                    'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', ...],
                              scaler=MinMaxScaler()))])

In [14]:
%%time
X_train_1 = prep_data_pipe_1.fit_transform(X_train, y_train)

CPU times: total: 6min 26s
Wall time: 6min 30s


In [15]:
X_train_1

TransactionDT  TransactionAmt         card1     card2     card3  \
0            0.175473        0.002716  0.000000e+00  0.001248  0.024507   
1            0.192424        0.015647  1.714286e-02  0.011070  0.024507   
2            0.455373        0.003655  0.000000e+00  0.016938  0.024507   
3            0.735004        0.003655  8.019246e-03  0.008080  0.024507   
4            0.812063        0.001526  7.837812e-08  0.022354  0.024507   
...               ...             ...           ...       ...       ...   
442900       0.131715        0.000774  8.573971e-05  0.022354  0.013514   
442901       0.389652        0.011199  2.097902e-02  0.021825  0.024507   
442902       0.571706        0.004876  1.562500e-02  0.022354  0.024507   
442903       0.160770        0.001839  3.608782e-02  0.031263  0.024507   
442904       0.147102        0.001369  3.808323e-02  0.041189  0.024507   

           card5     addr1     addr2     dist1  P_emaildomain  ...  \
0       0.029719  0.025430  0.034172  0.000000       0.023246  ...   
1       0.038068  0.039989  0.034172  0.000000       0.021653  ...   
2       0.056476  0.026195  0.034172  0.000000       0.021653  ...   
3       0.010743  0.026122  0.034172  0.000000       0.039052  ...   
4       0.029719  0.022745  0.034172  0.001458       0.039052  ...   
...          ...       ...       ...       ...            ...  ...   
442900  0.144319  0.191638  0.000232  0.000000       0.051749  ...   
442901  0.029719  0.025430  0.034172  0.000000       0.039052  ...   
442902  0.029719  0.016841  0.034172  0.000097       0.039052  ...   
442903  0.029719  0.026195  0.034172  0.000000       0.022512  ...   
442904  0.029719  0.025430  0.034172  0.000000       0.051749  ...   

        id_12_NotFound  id_15_New  id_15_Unknown  id_16_NotFound  id_28_New  \
0                    1          0              0               0          0   
1                    1          1              0               1          1   
2                    1          0              0               0          0   
3                    1          0              0               0          0   
4                    1          0              0               0          0   
...                ...        ...            ...             ...        ...   
442900               0          0              0               0          0   
442901               1          0              0               0          0   
442902               1          0              0               0          0   
442903               1          0              0               0          0   
442904               1          0              0               0          0   

        id_29_NotFound  id_35_T  id_36_T  id_37_T  id_38_T  
0                    0        1        0        1        0  
1                    1        1        0        1        1  
2                    0        1        0        1        0  
3                    0        1        0        1        0  
4                    0        1        0        1        0  
...                ...      ...      ...      ...      ...  
442900               0        1        0        1        1  
442901               0        1        0        1        0  
442902               0        1        0        1        0  
442903               0        1        0        1        0  
442904               0        1        0        1        0  

[442905 rows x 229 columns]

Отбор признаков с помощью SelectFromModel

In [17]:
%%time
sfm = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'), threshold=1e-2)
sfm.fit(X_train_1, y_train)

CPU times: total: 7min 39s
Wall time: 7min 42s


SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='liblinear'),
                threshold=0.01)

In [80]:
sfm.get_feature_names_out()

array(['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3',
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C3',
       'C5', 'C13', 'D1', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'V1',
       'V2', 'V3', 'V4', 'V6', 'V9', 'V10', 'V12', 'V15', 'V17', 'V19',
       'V23', 'V24', 'V25', 'V26', 'V27', 'V29', 'V35', 'V37', 'V38',
       'V39', 'V41', 'V42', 'V44', 'V46', 'V47', 'V49', 'V52', 'V53',
       'V55', 'V56', 'V59', 'V61', 'V62', 'V65', 'V66', 'V67', 'V70',
       'V72', 'V74', 'V75', 'V77', 'V78', 'V81', 'V82', 'V83', 'V86',
       'V87', 'V95', 'V98', 'V100', 'V104', 'V105', 'V107', 'V108',
       'V110', 'V112', 'V114', 'V115', 'V116', 'V118', 'V120', 'V121',
       'V123', 'V124', 'V135', 'V169', 'V170', 'V171', 'V172', 'V174',
       'V175', 'V176', 'V180', 'V181', 'V184', 'V185', 'V187', 'V189',
       'V191', 'V194', 'V199', 'V200', 'V207', 'V208', 'V209', 'V220',
       'V221', 'V223', 'V224', 'V228', 'V229', 'V238', 'V240', 'V241',
       'V242',

In [81]:
len(sfm.get_feature_names_out())

181

In [18]:
X_train = X_train_1[sfm.get_feature_names_out()]


Выполним предобработку тестовых данных

In [19]:
%%time
X_test = prep_data_pipe_1.transform(X_test)

CPU times: total: 54 s
Wall time: 54.2 s


In [20]:
X_test = X_test[sfm.get_feature_names_out()]

### LGBMClassifier с гиперпараметрами, подобранными с помощью optuna

In [21]:
%%time

# Лучшие результаты по кросс-валидации
# Первый:
# (0.8971254033794307,
#  {'n_estimators': 610,
#   'max_depth': 18,
#   'num_leaves': 90,
#   'eta': 0.045,
#   'reg_lambda': 5.8,
#   'subsample': 1.0,
#   'feature_fraction': 0.4})

# Второй:
# (0.8973130662047901,
#  {'n_estimators': 680,
#   'num_leaves': 62,
#   'eta': 0.061,
#   'reg_lambda': 3.7,
#   'subsample': 0.6000000000000001})

lgbm = LGBMClassifier(random_state=42,
                      n_estimators=680,
#                       max_depth=18,
                      num_leaves=90,
                      learning_rate=0.045,
                      reg_lambda=5.8,
#                       feature_fraction=0.4,
                      subsample=0.5,
                      class_weight='balanced',
                      verbose=0)

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112800 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
CPU times: total: 2min 52s
Wall time: 24.8 s


In [22]:
roc_auc_score(y_train, lgbm.predict(X_train))

0.9703210491342893

In [47]:
roc_auc_score(y_test, pred)

0.8993058968176928

In [26]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98    142330
           1       0.45      0.84      0.59      5305

    accuracy                           0.96    147635
   macro avg       0.72      0.90      0.78    147635
weighted avg       0.97      0.96      0.96    147635



### Подбор гиперпараметров с помощью optuna
Подберём оптимальные гиперпараметры с помощью optuna. Для подсчёта метрики roc_auc_score выполним кросс-валидацию по трём фолдам.

In [94]:
def lgbm_modelling(kfold, X_train, y_train, lgbm):

    skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    roc_aucs = []
    i = 1
    for train_idx, valid_idx in skf.split(X_train, y_train):
        trn_X = X_train.iloc[train_idx]
        trn_y = y_train.iloc[train_idx]
        val_X = X_train.iloc[valid_idx]
        val_y = y_train.iloc[valid_idx]
        
        clf = lgbm.fit(trn_X, trn_y, eval_set=[(val_X, val_y)], verbose=False)

        roc_aucs.append(roc_auc_score(val_y, clf.predict(val_X)))

        i += 1

    print('\nCV roc score        : {0:.4f}, std: {1:.4f}.'.format(np.mean(roc_aucs), np.std(roc_aucs)))
    return np.mean(roc_aucs)

In [121]:
def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=10)
#     max_depth = trial.suggest_int('max_depth', 6, 18)
    num_leaves = trial.suggest_int('num_leaves', 30, 100)
    eta = trial.suggest_float('eta', 0.001, 0.1, step=0.001)
    
    lambd = trial.suggest_float('reg_lambda', 0.0, 7.0, step=0.01)
    # alpha = trial.suggest_int('alpha', 0, 20)
    subsample = trial.suggest_float('subsample', 0.4, 1.0, step=0.1)
#     feature_fraction = trial.suggest_float('feature_fraction', 0.4, 1.0, step=0.1)

    lgbm = LGBMClassifier(n_estimators=n_estimators,
#                             max_depth=max_depth,
                            num_leaves=num_leaves,
                            learning_rate=eta,
                            reg_lambda=lambd,
                            # reg_alpha=alpha,
                            subsample=subsample,
#                             feature_fraction=feature_fraction,
                            class_weight='balanced',
                            random_state=42)

    return lgbm_modelling(3, X_train, y_train, lgbm)


In [122]:
%%time

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name="lgbm_clf", direction='maximize', sampler=TPESampler(seed=42), pruner=HyperbandPruner())
study.optimize(objective, n_trials=100)


CV roc score        : 0.8952, std: 0.0013.

CV roc score        : 0.8842, std: 0.0034.

CV roc score        : 0.8892, std: 0.0019.

CV roc score        : 0.8855, std: 0.0019.

CV roc score        : 0.8865, std: 0.0031.

CV roc score        : 0.8945, std: 0.0023.

CV roc score        : 0.8655, std: 0.0017.

CV roc score        : 0.8793, std: 0.0031.

CV roc score        : 0.8458, std: 0.0014.

CV roc score        : 0.8958, std: 0.0019.

CV roc score        : 0.8939, std: 0.0011.

CV roc score        : 0.8950, std: 0.0010.

CV roc score        : 0.8945, std: 0.0016.

CV roc score        : 0.8919, std: 0.0013.

CV roc score        : 0.8973, std: 0.0016.

CV roc score        : 0.8950, std: 0.0018.

CV roc score        : 0.8938, std: 0.0008.

CV roc score        : 0.8962, std: 0.0011.

CV roc score        : 0.8912, std: 0.0024.

CV roc score        : 0.8959, std: 0.0015.

CV roc score        : 0.8874, std: 0.0026.

CV roc score        : 0.8948, std: 0.0004.

CV roc score        : 0.8953, s

In [123]:
study.best_value, study.best_params

(0.8973130662047901,
 {'n_estimators': 680,
  'num_leaves': 62,
  'eta': 0.061,
  'reg_lambda': 3.7,
  'subsample': 0.6000000000000001})